In [1]:
import cv2
import face_recognition as fr
import os
import pickle
import numpy as np
import time
from multiprocessing import Pool


## Create Model

In [3]:
folder = 'Images'
imageNameList = os.listdir(folder)

names = []
imgList = []

for path in imageNameList:
    imgList.append(cv2.imread(os.path.join(folder, path)))
    names.append(os.path.splitext(path)[0])
    

In [4]:
def findEncodings(imgList):
    encodeList = []
    for img in imgList:
        print(img.shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        print(img.shape)
        encodeimg = fr.face_encodings(img)[0]
        encodeList.append(encodeimg)
    return encodeList


In [5]:
encodedListKnown = findEncodings(imgList)
encodedListKnownWithNames = [names, encodedListKnown]

(225, 225, 3)
(225, 225, 3)
(267, 189, 3)
(267, 189, 3)
(259, 194, 3)
(259, 194, 3)
(450, 333, 3)
(450, 333, 3)
(225, 225, 3)
(225, 225, 3)


Save Encodings

In [6]:
file = open('face_Encodings.p', 'wb')
pickle.dump(encodedListKnownWithNames, file)
file.close()

## Inferencing

In [7]:
file = open('face_Encodings.p', 'rb')
encodedListKnownWithNames = pickle.load(file)
file.close()
known_face_names, known_face_encodings = encodedListKnownWithNames

In [7]:
def process_frame(encoFace):
    print(encoFace.shape)
    print(encoFace.shape, faceloc)
    matches = fr.compare_faces(known_face_encodings, encoFace)
    faceDis = fr.face_distance(known_face_encodings, encoFace)
    matchIndex = np.argmin(faceDis)

    if matches[matchIndex]:
        recognized_name = known_face_names[matchIndex]
        print(recognized_name)
        # y1, x2, y2, x1 = faceloc
        # y1, x2, y2, x1 = y1 * 2, x2 * 2, y2 * 2, x1 * 2
        # boundingBox = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # cv2.putText(frame, recognized_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    else:
        print('Unknown')

In [8]:
def process_frame(encoFace):
    matches = fr.compare_faces(known_face_encodings, encoFace)
    faceDis = fr.face_distance(known_face_encodings, encoFace)
    matchIndex = np.argmin(faceDis)

    if matches[matchIndex]:
        recognized_name = known_face_names[matchIndex]
        print(recognized_name)
    else:
        print('Unknown')

In [8]:
cap = cv2.VideoCapture(0)
counter = 0
end_time = time.time()
fps = 0

while True:
    success, frame = cap.read()

    if not success:
        break

    if counter % 10 == 0:
        img = cv2.resize(frame, (0, 0), None, 0.5, 0.5)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        facesCurFrame = fr.face_locations(img)

        encodeCurFrame = fr.face_encodings(img, facesCurFrame)


    # Your code to calculate and display FPS here
    counter += 1
    if counter % 10 == 0:
        fps = 10 / (time.time() - end_time)  # Calculate FPS over the last 10 frames
        end_time = time.time()

        # Display the frame rate
        cv2.putText(frame, f'FPS: {round(fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [13]:
from collections import defaultdict

main_folder = os.path.join("../web_interface/API/static\\faces\\", "2023_10_31__22_36_37")
identified = defaultdict(list)
identified["base_folder"] = main_folder

for name in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, name)
    files = os.listdir(folder_path)
    sorted_files = sorted(files)
    first_5_files = sorted_files[:5]
    identified[name] = first_5_files


